In [1]:
!pip install textblob openai
import requests
import pandas as pd
from datetime import datetime
import json
import os
from textblob import TextBlob
from collections import Counter
from openai import OpenAI

In [3]:
# ---------- 1. Dataset Preparation ---------- #

def fetch_google_reviews(place_id, api_key):
    """Fetch reviews from Google Maps using SerpAPI."""
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "hl": "en",
        "api_key": api_key
    }
    response = requests.get("https://serpapi.com/search.json", params=params)
    reviews = response.json().get("reviews", [])
    data = []
    for r in reviews:
        data.append({
            "user": r.get("user"),
            "rating": r.get("rating"),
            "date": r.get("date"),
            "review_text": r.get("snippet")
        })
    df = pd.DataFrame(data)
    df.to_csv("google_reviews.csv", index=False)
    df.to_json("google_reviews.json", orient="records", indent=2)
    return df

In [4]:
# ---------- 2. Text Preprocessing ---------- #

def preprocess_reviews(df):
    """Clean and normalize review text."""
    df["review_text"] = df["review_text"].fillna("").str.lower()
    df["date"] = pd.to_datetime(df["date"], errors='coerce')
    df["year_month"] = df["date"].dt.to_period("M")
    return df


In [6]:
import os
import requests
import pandas as pd
from textblob import TextBlob
from openai import OpenAI

# =========================================================
# 1️⃣  FETCH GOOGLE MAPS REVIEWS FROM SERPAPI
# =========================================================
def fetch_google_reviews(place_id: str, api_key: str) -> pd.DataFrame:
    """Fetch live Google Maps reviews from SerpAPI."""
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "hl": "en",
        "api_key": api_key
    }
    r = requests.get("https://serpapi.com/search.json", params=params)
    data = r.json()
    reviews = data.get("reviews", [])
    if not reviews:
        print("⚠️  No reviews found. Check your API key or Place ID.")
        return pd.DataFrame()
    df = pd.DataFrame(
        {
            "user": [r.get("user", "Anonymous") for r in reviews],
            "rating": [r.get("rating") for r in reviews],
            "date": [r.get("date") for r in reviews],
            "review_text": [r.get("snippet", "") for r in reviews],
        }
    )
    return df


# =========================================================
# 2️⃣  PREPROCESS TEXT
# =========================================================
def preprocess_reviews(df: pd.DataFrame) -> pd.DataFrame:
    """Clean text and normalize date/time."""
    df["review_text"] = df["review_text"].fillna("").str.lower()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["year_month"] = df["date"].dt.to_period("M")
    return df


# =========================================================
# 3️⃣  SENTIMENT + TOPIC EXTRACTION
# =========================================================
def analyze_sentiment(text: str) -> str:
    """Classify sentiment using TextBlob."""
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "positive"
    elif polarity < -0.1:
        return "negative"
    return "neutral"


def extract_topics(text: str):
    """Keyword-based topic tagging."""
    topics = []
    keywords = {
        "service": ["service", "staff", "waiter", "employee"],
        "pricing": ["price", "expensive", "cheap", "value"],
        "ambience": ["ambience", "atmosphere", "environment"],
        "food": ["food", "menu", "dish", "taste"],
        "cleanliness": ["clean", "dirty", "hygiene"],
    }
    for topic, words in keywords.items():
        if any(w in text for w in words):
            topics.append(topic)
    return topics if topics else ["general"]


# =========================================================
# 4️⃣  SUMMARIZE VIA OPENAI / LANGFLOW
# =========================================================
def summarize_reviews(df: pd.DataFrame) -> str:
    """Summarize all customer feedback with LLM."""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    joined_text = " ".join(df["review_text"].tolist())[:4000]
    prompt = f"Summarize customer feedback into key insights:\n{joined_text}"
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a business analyst."},
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content.strip()


# =========================================================
# 5️⃣  MAIN EXECUTION
# =========================================================
if __name__ == "__main__":
    # 👉 Replace with your real credentials
    SERPAPI_KEY = os.getenv("SERP_API_KEY") or "3ba26c22be3acd9618c4f8d550ff38eb9b4775c2dc28a8c7458bd030ec038bc8" # Replace 'YOUR_SERP_API_KEY' with your actual SerpAPI key
    PLACE_ID = "ChIJK1DYbQNZqDsRFmLg734ccHU" # Replace 'YOUR_GOOGLE_PLACE_ID' with an actual Google Place ID (e.g., 'ChIJLU7jZClu5kcR4PcOOO6p3I0' for Louvre Museum)
    # Set your OpenAI API key here if not already set as an environment variable
    os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY" # <-- REPLACE THIS with your actual OpenAI API Key, for example, sk-proj-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

    print("📡 Fetching Google reviews...")
    df = fetch_google_reviews(PLACE_ID, SERPAPI_KEY)
    if df.empty:
        raise SystemExit("No reviews fetched — verify keys and Place ID.")

    print("🧹 Preprocessing reviews...")
    df = preprocess_reviews(df)

    print("🧠 Running sentiment and topic analysis...")
    df["sentiment"] = df["review_text"].apply(analyze_sentiment)
    df["topics"] = df["review_text"].apply(extract_topics)

    print("\n✅ Processed Reviews:")
    print(df[["user", "rating", "date", "sentiment", "topics", "review_text"]].head())

    # Optional summarization if you have an OpenAI key
    if os.getenv("OPENAI_API_KEY") and os.getenv("OPENAI_API_KEY") != "YOUR_OPENAI_API_KEY": # Added check for placeholder key
        print("\n💡 Generating summary with LLM...")
        summary = summarize_reviews(df)
        print("\n🧾 Summary of Customer Feedback:\n", summary)

    # Save results
    df.to_csv("google_reviews_processed.csv", index=False)
    print("\n📁 Saved results to 'google_reviews_processed.csv'")

📡 Fetching Google reviews...
🧹 Preprocessing reviews...
🧠 Running sentiment and topic analysis...

✅ Processed Reviews:
                                                user  rating date sentiment  \
0  {'name': 'xam tam', 'link': 'https://www.googl...     1.0  NaT  negative   
1  {'name': 'vignesh war', 'link': 'https://www.g...     1.0  NaT   neutral   
2  {'name': 'Mahendran Mani', 'link': 'https://ww...     4.0  NaT   neutral   
3  {'name': 'Rockstar Production', 'link': 'https...     1.0  NaT  positive   
4  {'name': 'Surya S', 'link': 'https://www.googl...     2.0  NaT   neutral   

                topics                                        review_text  
0            [service]  the senior doctor(the owner) and his son are t...  
1            [service]  my brother was discharged this morning, and du...  
2            [service]  the doctors are really very good and caring, w...  
3            [service]  they are saying 24 hours hospital service not ...  
4  [service, ambience]  m

/tmp/ipython-input-286777476.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


In [8]:
# ---------- 4. LLM Recommendation Generation ---------- #

def generate_recommendations(summary):
    """Ask LLM to generate improvement and trend recommendations."""
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    prompt = f"""
    Based on the following summary of reviews:
    {summary}
    Provide:
    1. Operational improvement suggestions.
    2. Trend-based business recommendations.
    3. Competitor comparison insights.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a business consultant."},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [ ]:
!pip install requests pandas transformers torch streamlit gradio textblob openai


In [ ]:
!pip install --upgrade --force-reinstall numpy transformers
import requests
import pandas as pd
from datetime import datetime
import json
from transformers import pipeline
from openai import OpenAI
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
def fetch_google_reviews(place_id: str, api_key: str) -> pd.DataFrame:
    """
    Fetch reviews using SerpAPI's Google Maps Reviews engine.
    """
    params = {
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "hl": "en",
        "api_key": api_key
    }
    response = requests.get("https://serpapi.com/search.json", params=params)
    reviews = response.json().get("reviews", [])
    data = []

    for r in reviews:
        data.append({
            "user": r.get("user"),
            "rating": r.get("rating"),
            "date": r.get("date"),
            "review_text": r.get("snippet")
        })

    df = pd.DataFrame(data)
    df.to_csv("google_reviews.csv", index=False)
    print(f"✅ Retrieved {len(df)} reviews.")
    return df

# Example usage:
# df = fetch_google_reviews("ChIJLU7jZClu5kcR4PcOOO6p3I0", "YOUR_SERP_API_KEY")

In [2]:
def preprocess_reviews(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean text and add time-related metadata.
    """
    df["review_text"] = df["review_text"].fillna("").str.lower()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["year_month"] = df["date"].dt.to_period("M")
    return df

# df = preprocess_reviews(df)


In [3]:
from transformers import pipeline
# Load Hugging Face model for sentiment analysis
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def classify_sentiment(text):
    """
    Classify sentiment (positive/negative/neutral) using Transformers.
    """
    if not text.strip():
        return "neutral"
    result = sentiment_model(text[:512])[0]
    label = result["label"].lower()
    return "positive" if "pos" in label else "negative"

# df["sentiment"] = df["review_text"].apply(classify_sentiment)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [4]:
def extract_topics(text):
    """
    Identify review topics using simple NLP keyword matching.
    """
    topics = []
    topic_map = {
        "service": ["service", "staff", "waiter", "helpful", "employee"],
        "pricing": ["price", "cost", "expensive", "cheap", "value"],
        "ambience": ["ambience", "atmosphere", "environment"],
        "food": ["food", "menu", "taste", "dish", "meal"],
        "cleanliness": ["clean", "hygiene", "tidy"]
    }
    for t, words in topic_map.items():
        if any(word in text for word in words):
            topics.append(t)
    return topics if topics else ["general"]

# df["topics"] = df["review_text"].apply(extract_topics)


In [5]:
def summarize_reviews(df: pd.DataFrame):
    """
    Use an OpenAI model to summarize all reviews.
    """
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    combined_text = " ".join(df["review_text"].tolist())[:4000]
    prompt = f"Summarize customer feedback into 3 bullet points: trends, praise, and complaints.\n\n{combined_text}"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert market analyst."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# summary = summarize_reviews(df)
